# Brain Stroke Detection

### ToDo
- [x] Download Dataset
- [ ] Exploratory Data Analysis
- [ ] Data Preprocessing
- [ ] Training Model
- [ ] HyperTuning
- [ ] Evaluating

### Other Ideas
- [ ] Feature Engineer

In [1]:
import os

os.listdir()

['dataset',
 '.gitignore',
 'README.md',
 '.kaggle',
 '.git',
 'Brain_Stroke_Detection.ipynb',
 'requirements.txt',
 'modified_dataset']

In [9]:
# Download Dataset

import os
import opendatasets as od

dataDir = os.listdir(path = './dataset')

DATA_URL = 'https://www.kaggle.com/datasets/jillanisofttech/brain-stroke-dataset'

def downloadDataset(url = DATA_URL):
    if 'brain-stroke-dataset' in dataDir:
        print("Dataset Already There ...")
    else:
        od.download(DATA_URL, data_dir = './dataset/')


downloadDataset()

100%|██████████| 47.2k/47.2k [00:00<00:00, 1.16MB/s]

In [10]:
# Exploratory Data Analysis

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt